<a href="https://colab.research.google.com/github/anandhc6/Assignment-3/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Required packages
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend
from random import randrange 
from tensorflow import keras
from google.colab import files
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.callbacks import EarlyStopping

In [ ]:
# Downloading dataset

!wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar
!tar -xf 'dakshina_dataset_v1.0.tar'